In [1]:
import sympy as sp
from IPython.display import display, Math
import time
start_time = time.time()

In [2]:
# x, y sú súradnice bodov v R^2
x, y = sp.symbols('x y')
# t je parameter z intervalu I
t = sp.Symbol('t')
# a, b sú konštanty pre škálovanie elipsoidu, a - dotykový smer, b - normálový smer
a, b = sp.symbols('a b')
# všetky ostatné symboly sú parametre v parametrizácii krivky parametric_curve
c, d = sp.symbols('c d')

In [3]:
# Vstupné parametre
# interval vyčíslenia krivky
t_min = -10
t_max = 10
# parametre škálovania
a_0 = 2
b_0 = 1
# parametre parametrizácie
c_0 = 2
d_0 = 1

# parametrizácia krivky
parametric_curve = sp.Matrix([c_0*t, d_0*t]) 
curve_name = 'line'

In [4]:
# Vypočíta dotyčnicu a normálu pre parametric_curve
def local_coordinate_system(parametric_curve): 
    t = sp.Symbol('t') 
    parametric_curve_first_derivation = sp.diff(parametric_curve, t)
    T = parametric_curve_first_derivation / sp.sqrt(parametric_curve_first_derivation.dot(parametric_curve_first_derivation))
    T = sp.simplify(T)
    N = sp.Matrix([-T[1], T[0]])
    N = sp.simplify(N)
    return T, N

# Vyrobí inverznú maticu prechodu 
def transition_matrix(T, N):
    P = sp.Matrix.hstack(T, N)
    return P.T

def matrix_of_quadrics(A, B, C, D, E, F):
    row1 = [A, B/2, D/2]
    row2 = [B/2, C, E/2]
    row3 = [D/2, E/2, F]
    matrix = sp.Matrix([row1, row2, row3])
    return matrix

In [5]:
# Výpočet vektorov
T, N = local_coordinate_system(parametric_curve)
# Zostavenie matice
P = transition_matrix(T, N)
# Maticou P vynásobíme vektor (X-parametric_curve) a tak zmeníme bázu elipsy
vector = sp.Matrix([x - parametric_curve[0], y - parametric_curve[1]])
result_vector = P * vector
ellipse_in_new_basis = result_vector[0]**2/a**2 + result_vector[1]**2/b**2 - 1
# Derivácia jednoparametrického systému elíps 
derivation = sp.diff(ellipse_in_new_basis, t)

In [6]:
# Výpočet, o aký typ kužeľosečky ide
# Najprv potrebujeme extrahovať koeficienty z derivácie
expanded_expression = sp.expand(derivation)
A = expanded_expression.coeff(x**2)
B = expanded_expression.coeff(x*y)
C = expanded_expression.coeff(y**2)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x*y, x*z,..
coef_x = expanded_expression.coeff(x)
terms_x = sp.Add(*coef_x.args) if isinstance(coef_x, sp.Add) else coef_x
terms_x_filtered = [term for term in sp.Add.make_args(terms_x) if not term.has(y)]
D = sp.Add(*terms_x_filtered)

coef_y = expanded_expression.coeff(y)
terms_y = sp.Add(*coef_y.args) if isinstance(coef_y, sp.Add) else coef_y
terms_y_filtered = [term for term in sp.Add.make_args(terms_y) if not term.has(x)]
E = sp.Add(*terms_y_filtered)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x, y, z
terms = sp.Add(*expanded_expression.args) if isinstance(expanded_expression, sp.Add) else expanded_expression
terms_filtered = [term for term in sp.Add.make_args(terms) if not term.has(x) and not term.has(y)]
F = sp.Add(*terms_filtered)

matrix_of_derivation = matrix_of_quadrics(A, B, C, D, E, F)

# Print koeficientov
print("Coefficient of x^2:")
display(Math(sp.latex(sp.simplify(A))))
print("Coefficient of y^2:")
display(Math(sp.latex(sp.simplify(C))))
print("Coefficient of xy:") 
display(Math(sp.latex(sp.simplify(B))))
print("Coefficient of x:")
display(Math(sp.latex(sp.simplify(D))))
print("Coefficient of y:")
display(Math(sp.latex(sp.simplify(E))))
print("Coefficient of constant term:")
display(Math(sp.latex(sp.simplify(F))))

Coefficient of x^2:


<IPython.core.display.Math object>

Coefficient of y^2:


<IPython.core.display.Math object>

Coefficient of xy:


...

<IPython.core.display.Math object>

Coefficient of y:


<IPython.core.display.Math object>

Coefficient of constant term:


<IPython.core.display.Math object>

In [7]:
# Typ kužeľosečky
determinant = sp.det(matrix_of_derivation)
subdeterminant = A*C-B**2/4

In [8]:
# Ukončenie merania času
end_time = time.time()
elapsed_time = end_time - start_time

In [9]:
# Typ kuželosečky vieme jasne dourčiť pre špecifikáciu parametrov
t_values = range(t_min, t_max)
is_negative = True

for t_val in t_values:
    if subdeterminant.subs({t: t_val, a: a_0, b: b_0, c: c_0, d: d_0}) >= 0:
        is_negative = False
        break

if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Výraz nie je pre zadané hodnoty záporný. 


In [11]:
print(parametric_curve)
print("Systém: ")
system = sp.latex(sp.simplify(ellipse_in_new_basis))
display(Math(system))

Matrix([[2*t], [t]])
Systém: 


<IPython.core.display.Math object>

In [12]:
print("Derivácia sytému: ")
der = sp.latex(sp.simplify(derivation))
display(Math(der))

Derivácia sytému: 


<IPython.core.display.Math object>

In [13]:
print("Determinant derivácie: ")
display(Math(sp.latex(sp.simplify(determinant))))

Determinant derivácie: 


<IPython.core.display.Math object>

In [ ]:

print("Čas: ")
print(elapsed_time, "seconds")

expression = sp.latex(sp.simplify(subdeterminant))
display(Math(expression))

In [11]:
with open(curve_name + '.txt', "w") as file:
    # Write content to the file
    file.write(f"{curve_name} {parametric_curve}\n")
    file.write('System:\n')
    file.write(f"{system} = 0\n")
    file.write('Derivation:\n')
    file.write(f"{der} = 0\n")
    file.write('Time:\n')
    file.write(f"{elapsed_time}\n")
    file.write('Determinant:\n')
    file.write(f"{determinant}\n")
    file.write('Subdeterminant:\n')
    file.write(f"{expression}\n")